<a href="https://colab.research.google.com/github/cheyroseflammer/big-data-challenge/blob/main/gift_card.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
 import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ 

In [12]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-03 18:29:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-02-03 18:29:43 (10.4 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [15]:
from pyspark import SparkFiles
# Load in book data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').option("sep", "\t").csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [18]:
# Create customer dataframe
customer_df = df.select(["customer_id","product_category","review_id"])\
  .groupby("customer_id","product_category")\
  .agg({"review_id":"count"})

customer_df = customer_df.withColumnRenamed("count(review_id)","customer_count")
customer_df.show(truncate=False)

+-----------+----------------+--------------+
|customer_id|product_category|customer_count|
+-----------+----------------+--------------+
|1094807    |Gift Card       |2             |
|13288873   |Gift Card       |1             |
|28907327   |Gift Card       |1             |
|152204     |Gift Card       |1             |
|12374492   |Gift Card       |1             |
|11804769   |Gift Card       |1             |
|11218637   |Gift Card       |1             |
|22047574   |Gift Card       |1             |
|36506397   |Gift Card       |1             |
|49927353   |Gift Card       |1             |
|754173     |Gift Card       |1             |
|15896710   |Gift Card       |1             |
|41292650   |Gift Card       |1             |
|33902981   |Gift Card       |1             |
|5126816    |Gift Card       |1             |
|47279457   |Gift Card       |1             |
|3440033    |Gift Card       |1             |
|47515165   |Gift Card       |1             |
|216088     |Gift Card       |1   

In [20]:
# Creating product dataframe
product_df = df.select('product_id','product_title').distinct()
product_df.show(truncate=False)

+----------+--------------------------------------------------------------------------------------------+
|product_id|product_title                                                                               |
+----------+--------------------------------------------------------------------------------------------+
|B00CHQKNUM|Amazon.com Gift Card in a Black Gift Box (Previous Generation Card Designs)                 |
|B00CRQ4ZOM|Amazon Gift Card - Print - #1 Dad                                                           |
|B00H5BMZ32|Amazon eGift Card - Eyes Know What You Did (Animated) [Hallmark]                            |
|BT00CTP84S|Amazon.com Gift Card in a Greeting Card (Various Designs)                                   |
|B004LLILN2|Amazon eGift Card - Happy Mother's Day (Child Art)                                          |
|B00N59437O|Amazon Gift Card - Email - Birthday Birthstone: January (Garnet)                            |
|B005J0444E|Amazon.com Gift Card in a Greeting

In [21]:
from pyspark.sql.types import DateType

# Create review df
review_df = df.select('review_id','customer_id','product_id','product_parent','product_category','review_date')
review_df=review_df.withColumn('review_date',review_df['review_date'].cast(DateType()))

review_df.show()

+--------------+-----------+----------+--------------+----------------+-----------+
|     review_id|customer_id|product_id|product_parent|product_category|review_date|
+--------------+-----------+----------+--------------+----------------+-----------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806|       Gift Card| 2015-08-31|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287|       Gift Card| 2015-08-31|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283|       Gift Card| 2015-08-31|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283|       Gift Card| 2015-08-31|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939|       Gift Card| 2015-08-31|
|R3ALA9XXMBEDZR|   18513645|B004KNWWU4|     326384774|       Gift Card| 2015-08-31|
|R3R8PHAVJFTPDF|   22484620|B004LLIKVU|     473048287|       Gift Card| 2015-08-31|
|R18WWEK8OIXE30|   14765851|BT00CTP2EE|     775486538|       Gift Card| 2015-08-31|
|R1EGUNQON2J277|   18751931|B004LLIKVU|     473048287|       Gift Card| 2015

In [23]:
# Creating vine df
vine_df = df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|
| RNYLPX611NB7Q|          5|            0|          0|   N|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|
|R18WWEK8OIXE30|          5|            0|          0|   N|
|R1EGUNQON2J277|          1|            0|          0|   N|
|R21Z4M4L98CPU2|          5|            0|          0|   N|
| R6JH7A117FHFA|          5|            0|          0|   N|
|R1XZHS8M1GCGI7|          5|            1|          1|   N|
|R1DAI0N03SKRJN|          5|            1|          1|   N|
|R2F6SKZOEYQRU3|          5|            

In [24]:
# Configure RDS
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/big_data_db"
config = {"user":"root", 
          "password": "<insert password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write data to RDS tables
customer_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
product_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
review_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)